# Loading fMRI Data Files into Python 

For this script we will use a localizer dataset from [Kim et al. (2017)](http://www.jneurosci.org/content/early/2017/01/23/JNEUROSCI.3272-16.2017). For the localizer, subjects were shown faces, scenes and objects in 3 different runs.

### Goal of this script
Using this script you will learn to load the fMRI data into python, plot a timeseries for a voxel, and Z-score the data -- an important normalization step for machine learning. General fMRI preprocessing, such as motion correction and temporal detrending, is also important, but has already been applied to these data. You will also learn to use Bokeh, a package for plotting, that will help you explore your data.


### Import necessary packages.
The following packages will be used:
            1. nibabel: Read  fMRI data into python arrays.  
            2. numpy: Perform numerical operations in python. 
            3. scipy: Scientific computing methods.  
            4. nilearn: Used to extract masked fMRI data from a Nifti file.
            5. sklearn: Machine Learning methods.  
            6. bokeh: Plotting library.  

In [1]:
import nibabel as nib
import numpy as np
from nilearn.input_data import NiftiMasker,  MultiNiftiMasker
import scipy.io
from scipy import stats
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from sklearn import preprocessing

output_notebook()

Loading BokehJS ...

**Exercise:** Describe the difference in functionality between 'import numpy' and 'from numpy import zeros':

"import numpy" imports all numpy modules and functions. By contrast, "from numpy import zeros" imports only the "zeros" function. Other modules would have to be loaded separately.


### Load in timing files

The first way we are going to examine this dataset, heretofore referred to as the "vdc" dataset, is by looking at the timing of events in the experiment. The labels for each run of the experiment are stored in the localizer matlab file (e.g., sub-01_localizer_01.mat). We will read and plot the data that show which stimulus was presented at what time during the experiment. The columns in the data represent time in the experiment.

**Self-study:** Navigate through the folder that contains these data (defined in 'dir' below) to get an understanding of the file structure. Open up the files and look at their contents. Be an explorer!

**Exercise:** How big is the dataset in terms of number of subjects and total file size?

From looking at the directory named below, there are 32 subjects (in folders named sub-01 to sub-32). Running a "du" Unix command in that directory shows that the toal file size is 98,380,800 KB, or 98.4 GB.


In [2]:
dir = '/gpfs/milgram/data/cmhn-s18/datasets/vdc/'
sub = 'sub-01';
stim_label = [];
stim_label_allruns = [];
for run in range(1,4):
    in_file= (dir + sub + '/ses-day2/design_matrix/' + "%s_localizer_0%d.mat" % (sub, run))
 
    # Load in data from matlab
    stim_label = scipy.io.loadmat(in_file);
    stim_label = np.array(stim_label['data']);
    
    # Store the data
    if run == 1:
             stim_label_allruns = stim_label;
    else:       
             stim_label_allruns = np.hstack((stim_label_allruns, stim_label))

print("Data Loading Completed")

Data Loading Completed


### Data File Description
The localizer consists of 3 runs with 5 blocks for each category. In the matlab stimulus file that we just loaded there is a bunch of information. Each row contains a different type of information (e.g., stimulus type, reaction time, etc.) and each column represents a different trial. 

Of most importance to us, the first row has the stimulus labels for the localizer runs; whereas the third row contains the time in seconds when the stimulus was presented (relative to the start of each run). The data were loaded in such that the three runs were concatenated in time.

The stimulus labels and their corresponding categories are as follows:  
1= Faces  
2= Scenes  
3= Objects  

### Let's plot the stimulus presentation for one run.

We prep the data for plotting. First we set different colors for each stimulus type. We also assign category labels for each time-point (e.g., 'Faces'). The plotting package Bokeh uses the ColumnDataSource format for easy color, label, and legend management for the figure.

In [3]:
# Create a dictionary with a key and a code 
colormap = {1: "red",2: "green", 3: 'blue'}
label_key = {1: "Faces", 2: "Scenes", 3: "Objects"}

# Cycle through each trial to determine the appropriate label
final_color=[]
final_label_list=[]
for x in (stim_label_allruns[0, :]):
    lab=(int(x))
    colors = colormap[lab] 
    labs_list = label_key[lab]
    final_color = np.hstack((final_color, colors))
    final_label_list= np.hstack((final_label_list, labs_list))

**Self-study:** Don't know what a dictionary is? Look it up!


**Exercise:** Loops are bad and should be avoided whenever possible. Try rewrite this code without a loop.

In [4]:
lab = (stim_label_allruns[0,:])
lab = lab.astype(int)

def getInd(ind, dic):
    return dic[ind]
vecGetInd = np.vectorize(getInd)

final_colors = vecGetInd(lab, colormap)
final_labs_list = vecGetInd(lab, label_key)

In [5]:
# Define the mapping between the keys and the values
data_key = dict(x = stim_label_allruns[2, 0:150],
                y = stim_label_allruns[0, 0:150],
                color = final_color[0:150],
                labels = final_label_list[0:150])
source = ColumnDataSource(data_key)

In [6]:
# Now we plot the data using Bokeh.
p = figure(title="Stimulus Presentation for Run 1")
p.circle(x='x', 
         y='y', 
         size=10, 
         color='color', 
         legend = 'labels',  
         source=source)
p.line(x = stim_label_allruns[2,0:150],
       y = stim_label_allruns[0,0:150],
      )
p.xaxis.axis_label = 'Time (seconds)'
p.yaxis.axis_label = 'Stimulus Label'
p.legend.location= "bottom_left"
show(p)

**Exercise:** Look up how to save this plot to your repo on the cluster and do so. It will become part of your submission.

**Exercise:** Plot the stimulus presentation for runs 2 and 3 for this subject.

**Exercise:** How many stimuli were presented in each block?

In each run, there were 15 blocks (as can be seen by the number of distinct stretches in the figure). There are 150 labels given per block. 150/15 = 10, so there are 10 stimuli per block.

**Exercise:** Is the stimulus presented in the same order for all the three runs?

No, we can see from the figures that the order of colors (ie classes) is different in each run, presumably randomized.

In [7]:
run_trials = 150
for i in range(2,4):
    data_key = dict(x = stim_label_allruns[2, (i-1)*run_trials+1:i*run_trials],
                y = stim_label_allruns[0, (i-1)*run_trials+1:i*run_trials],
                color = final_color[(i-1)*run_trials+1:i*run_trials],
                labels = final_label_list[(i-1)*run_trials+1:i*run_trials])
    source = ColumnDataSource(data_key)
    p = figure(title="Stimulus Presentation for Run " + str(i))
    p.circle(x='x', 
             y='y', 
             size=10, 
             color='color', 
             legend = 'labels',  
             source=source)
    p.line(x = stim_label_allruns[2,(i-1)*run_trials+1:i*run_trials],
           y = stim_label_allruns[0,(i-1)*run_trials+1:i*run_trials],
          )
    p.xaxis.axis_label = 'Time (seconds)'
    p.yaxis.axis_label = 'Stimulus Label'
    p.legend.location= "bottom_left"
    show(p)

### Loading the fMRI data.

We'll load the data for one run. We will also extract a subset of the signal from the whole-brain data by using a mask for the "fusiform face area" ('FFA'). Each run has approximately 310 TRs.

In [8]:
# Load data files for one run of the localizer.
maskdir = (dir + sub + "/preprocessed/masks/")
masks = ['FFA']

# Cycle through the masks
for mask in masks:
    print ("Processing Start ...")
    maskfile = (maskdir + "sub-01_ventral_%s_locColl_to_epi1.nii.gz" % (mask))
    mask = nib.load(maskfile)
    print ("Loaded Mask")
    print(mask.shape)
    
    # Cycle through the runs
    for run in range(1,2):
        epi_in = (dir + sub + "/preprocessed/loc/%s_filtered2_d1_firstExampleFunc_r%d.nii" % ( sub,run))
        epi_data = nib.load(epi_in)
        nifti_masker = NiftiMasker(mask_img=mask)
        epi_mask_data = nifti_masker.fit_transform(epi_data); 



Processing Start ...
Loaded Mask
(128, 128, 52)


**Self-study:** What is the parahippocampal placea area (PPA)?

**Exercise:** Make a volume with a PPA mask and create an image to show a slice through this mask.

**Exercise:** We used nitime to do the masking in the above code. How else could you mask the fMRI data?

You could make a 3D array of the same dimensions as your brain data. Values would be 1 wherever we wanted to consider it PPA and 0 everywhere else. 

In [9]:
# Load data files for one run of the localizer.
maskdir = (dir + sub + "/preprocessed/masks/")
masks = ['PPA']

# Cycle through the masks
for mask in masks:
    print ("Processing Start ...")
    maskfile = (maskdir + "sub-01_ventral_%s_locColl_to_epi1.nii.gz" % (mask))
    mask = nib.load(maskfile)
    print ("Loaded Mask")
    print(mask.shape)
    
    # Cycle through the runs
    for run in range(1,2):
        epi_in = (dir + sub + "/preprocessed/loc/%s_filtered2_d1_firstExampleFunc_r%d.nii" % ( sub,run))
        epi_data = nib.load(epi_in)
        nifti_masker = NiftiMasker(mask_img=mask)
        epi_mask_data = nifti_masker.fit_transform(epi_data); 

Processing Start ...
Loaded Mask
(128, 128, 52)


In [10]:
mask_data = mask.get_data()
coronal_plane = mask_data[:,int(mask_data.shape[1] / 2), :]
p = figure(title="PPA Mask",plot_width=400, plot_height=400, \
               x_range=(0, mask.shape[0]), y_range=(0, mask.shape[1]))
p.axis.visible = False
p.image(image=[coronal_plane], x=[0], y=[0], dw=[mask.shape[0]], dh=[mask.shape[1]])
show(p)

### Plot a voxel time-series

After masking, the fMRI dataset at this stage is in the format rows=time x columns=voxels. First, we'll transpose the data to make the time series go across columns. Each time-point will be a column vector of voxel values.   

In [11]:
# What is the dimensionality of the data
print(epi_mask_data.shape)

maskedData = np.transpose(np.copy(epi_mask_data))
print(maskedData.shape)

(310, 2504)
(2504, 310)


In [12]:
# Plot a voxel value through time
voxel_num = 100
p2 = figure(title="Voxel time-series Run 1 for voxel number = %d" % (voxel_num))
p2.line(x = range(1, 311),y = maskedData[0:310,voxel_num])
p2.xaxis.axis_label = 'TR'
p2.yaxis.axis_label = 'Voxel Intensity'

show(p2)

## Scaling the data a.k.a Z-scoring

Now we'll scale the data to center the mean around zero and have a standard deviation of one ($\mu=0, \sigma = 1$) for  the data. Why do we scale the data? See here: http://scikit-learn.org/stable/modules/preprocessing.html

There are many ways to scale the data. Z-scoring is one of the most common approaches. We will use the StandardScaler method to accomplish this.


**Self-study:** Explore other normalization techniques in the link above.

In [13]:
scaler = preprocessing.StandardScaler().fit(maskedData)

maskedData_zscore =  scaler.transform(maskedData)                               

### Check if the z scoring worked

The mean values never equal exactly zero. This happens because of rounding and precision limitations. These small values are considered zero for most practical purposes. We can print out the values. Also check that the standard deviation is now correct.

In [14]:
x_mean = np.mean(maskedData_zscore, axis=0)
x_std = np.std(maskedData_zscore, axis=0)
print(x_mean[0:10])
print(x_std[0:10])

[  6.09376549e-09   1.58437899e-07  -1.40156615e-07   5.27110728e-07
   4.57032421e-08   5.48438912e-08  -2.52891283e-07  -4.23516695e-07
   1.64531670e-07  -3.65625930e-08]
[ 1.00000012  1.          1.          1.          1.          0.99999988
  1.          0.99999994  1.          1.        ]


**Exercise:** Plot the distribution of values as a histogram.

In [15]:
hist = figure(title="Histogram of Intensity Values for Voxel 100 Over Time")
dat, edges = np.histogram(maskedData_zscore[:,100], bins=40)
hist.quad(top=dat, bottom = 0, 
         left = edges[:-1], right=edges[1:])
show(hist)

In [16]:
row_num = 100 # Choose a time-point to plot a feature column
p3 = figure(title="Scaled Voxel values for time point = %d" % (row_num))
p3.line(x=range(1,5520),y= maskedData_zscore[row_num,:]) # Plotting the column here instead of rows.
p3.xaxis.axis_label = 'Voxel Number'
p3.yaxis.axis_label = 'Voxel Intensity (Standardized)'

show(p3)

**Exercise:** Z-score the data by writing your own code instead of using the StandardScaler() method


In [17]:
# Insert your code here

# Compute the mean of the data
# Compute the standard deviation
maskedData_z = np.empty(maskedData.shape)
voxelMeans = np.mean(maskedData, axis=1)
voxelSD = np.std(maskedData, axis=1)

#print(voxelMeans.shape) = (2504,)
#print(maskedData.shape) = (2504, 310)
#print(maskedData_z.shape) = (2504, 310)

# Z-score the data

for i in range(maskedData.shape[1]):
    maskedData_z[:,i] = np.subtract(maskedData[:,i], voxelMeans)
    maskedData_z[:,i] = np.divide(maskedData_z[:,i], voxelSD)

x_mean = np.mean(maskedData_z, axis=1)
x_std = np.std(maskedData_z, axis=1)

print(x_mean[0:10])
print(x_std[0:10])


# Plot the results for the a feature vector at one time-point

row_num = 200 # Choose a time-point to plot a feature column
p4 = figure(title="Scaled Voxel values for time point = %d" % (row_num))
p4.line(x=range(1,5520),y= maskedData_z[row_num,:]) # Plotting the column here instead of rows.
p4.xaxis.axis_label = 'Voxel Number'
p4.yaxis.axis_label = 'Voxel Intensity (Standardized)'

show(p4)

# Plot the time-series for one voxel

voxel_num = 200
p5 = figure(title="Voxel time-series Run 1 for voxel number = %d" % (voxel_num))
p5.line(x = range(1, 311),y = maskedData_z[0:310,voxel_num])
p5.xaxis.axis_label = 'TR'
p5.yaxis.axis_label = 'Voxel Intensity'

show(p5)

[  1.29097290e-06  -1.67334082e-05  -1.02199509e-05  -2.28716352e-05
   5.35460210e-07   2.72224666e-05  -1.82192063e-06  -3.00883994e-06
  -5.59395522e-06   2.13057993e-06]
[ 1.00000006  0.99999983  1.00000012  1.00000015  0.9999999   1.00000021
  1.00000007  1.00000004  0.99999993  1.00000023]


**Novel contribution:** be creative and make one new discovery by adding an analysis, visualization, or optimization.

Make a histogram plotting the standard deviation of each (non-z-scored) voxel to see which had the greatest variability in firing over the course of the run.

In [18]:
voxelSD = np.std(maskedData, axis=1)
hist = figure(title="Standard Deviation of Voxels in PPA")
dat, edges = np.histogram(voxelSD, bins=40)
hist.quad(top=dat, bottom = 0, 
         left = edges[:-1], right=edges[1:])
show(hist)
